In [1]:
import json
import pandas as pd
from datetime import datetime as dt
from datetime import datetime, timedelta
import time
import csv
import statsmodels.api as sm
import os
import numpy as np
import math
import requests


In [7]:
df=pd.read_csv('Raw Data/pratham opening.csv')
df['side']='open'
df

,pair,price,time,trader,position_side,fee,volume,token_amount,price_into_token_amount,side
0,wETH,1338.700,2023-01-10 20:37:19.000 UTC,0x8d6d43e62c30df4b3b270978316ea8c8a8daffad,SHORT,6.469667,5711.638139,0.294563,394.332090,open
1,wBTC,22683.000,2023-01-23 15:05:37.000 UTC,0x402030430bd2aea98edb036736db146c529aede0,SHORT,238.095238,238095.238006,0.220429,5000.000000,open
2,wBTC,21302.320,2022-07-26 00:03:47.000 UTC,0x35a9ebf9d86cfead962494eddab25ac8b640e9c9,LONG,122.716933,21789.725080,0.233495,4973.995220,open
3,wETH,1534.770,2023-03-09 00:38:38.000 UTC,0x71889f7c734342741708e9f1e0d86874c91eacaf,LONG,14.909535,14909.535067,0.648826,995.799348,open
4,wBTC,27246.659,2023-06-20 16:43:03.000 UTC,0x9031b0853bec292701053cea627b2ff4f22249bb,LONG,14.238571,14238.571265,0.010974,298.995572,open
...,...,...,...,...,...,...,...,...,...,...
18959,wETH,1578.200,2023-02-25 20:25:12.000 UTC,0xe786ec5da1c8b391f896d78791091554bc0464f2,LONG,47.452381,47452.380952,0.631415,996.499602,open
18960,wBTC,27188.400,2023-06-20 16:18:35.000 UTC,0x35a9ebf9d86cfead962494eddab25ac8b640e9c9,SHORT,48.526211,48526.210757,0.054884,1492.221739,open
18961,wETH,2968.235,2022-04-22 14:26:44.000 UTC,0xa9a37b387b0017fbcb4b0c99078445f070703024,LONG,10.966340,10098.147047,0.250000,742.058750,open
18962,wETH,1847.233,2023-05-09 16:55:30.000 UTC,0x4e839f41077cf5aece743e9cca9b26ffe450d712,LONG,15.481982,15481.981918,0.900000,1662.509700,open


In [8]:
df2=pd.read_csv('Raw Data/pratham closing.csv')
df2['side']='close'
df2.rename(columns = {'virtual_asset':'pair'}, inplace = True)

df2

,pair,price,time,trader,volume,fee,pnl,pnl_percent,position_side,liquidation,return,pnltocollateral,token_amount,price_into_token_amount,side
0,wETH,1333.300,2022-09-29 17:50:32.000 UTC,0x289a43b5c3cb41dbdef69d0b301915df56af3623,1.343000e+04,14.074640,248.630274,1.851305,LONG,No,0.218728,0.075,2.190287,2920.310029,close
1,wETH,1613.220,2023-01-29 01:22:27.000 UTC,0x289a43b5c3cb41dbdef69d0b301915df56af3623,3.812549e+04,38.125490,865.306194,2.269626,LONG,No,0.761238,0.075,2.897334,4674.036455,close
2,wETH,1646.990,2023-03-02 05:38:32.000 UTC,0x289a43b5c3cb41dbdef69d0b301915df56af3623,3.100001e+04,32.147008,192.458964,0.620835,SHORT,No,0.169312,0.075,0.719424,1184.883880,close
3,wETH,1089.900,2022-06-17 20:40:09.000 UTC,0xa9a37b387b0017fbcb4b0c99078445f070703024,3.005672e+04,30.056718,175.465654,0.583782,SHORT,No,0.092935,0.041,1.353658,1475.352217,close
4,wETH,1269.530,2022-11-12 10:17:34.000 UTC,0xa9a37b387b0017fbcb4b0c99078445f070703024,7.487485e+04,81.014588,-1252.061732,-1.672206,SHORT,No,-0.663153,0.041,1.830118,2323.389623,close
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9917,wBTC,30183.908,2023-04-11 13:55:35.000 UTC,0x7b076ddcb2abe2c9c167b7f4f57e18b5f5eb1c95,4.318619e+04,51.780242,-147.078592,-0.340569,SHORT,No,-0.412810,0.097,0.124933,3770.954976,close
9918,wBTC,17486.380,2023-01-10 20:21:21.000 UTC,0x10ee2ce2d6f0544ffa8222ca13878896e8124bad,2.862495e+05,326.324381,-4375.138125,-1.528435,SHORT,No,-11.383663,0.408,0.137138,2398.048264,close
9919,wETH,1871.830,2023-05-03 17:57:53.000 UTC,0x1c3ce81f49e2d4203bd7fa5f9c080159d566d28d,5.965176e+06,5965.175890,47898.179997,0.802963,SHORT,No,1.370841,0.265,114.555231,214427.918101,close
9920,wETH,1909.701,2023-05-03 22:17:24.000 UTC,0x1c3ce81f49e2d4203bd7fa5f9c080159d566d28d,1.479214e+07,15798.008144,100812.234636,0.681526,LONG,No,2.885236,0.265,201.647571,385086.568035,close


In [9]:
df3=df2.drop(['pnl', 'pnl_percent', 'return', 'liquidation', 'pnltocollateral'], axis=1)

df3['position_side'] = df2['position_side'].replace({'LONG': 'SHORT', 'SHORT': 'LONG'})
df3

,pair,price,time,trader,volume,fee,position_side,token_amount,price_into_token_amount,side
0,wETH,1333.300,2022-09-29 17:50:32.000 UTC,0x289a43b5c3cb41dbdef69d0b301915df56af3623,1.343000e+04,14.074640,SHORT,2.190287,2920.310029,close
1,wETH,1613.220,2023-01-29 01:22:27.000 UTC,0x289a43b5c3cb41dbdef69d0b301915df56af3623,3.812549e+04,38.125490,SHORT,2.897334,4674.036455,close
2,wETH,1646.990,2023-03-02 05:38:32.000 UTC,0x289a43b5c3cb41dbdef69d0b301915df56af3623,3.100001e+04,32.147008,LONG,0.719424,1184.883880,close
3,wETH,1089.900,2022-06-17 20:40:09.000 UTC,0xa9a37b387b0017fbcb4b0c99078445f070703024,3.005672e+04,30.056718,LONG,1.353658,1475.352217,close
4,wETH,1269.530,2022-11-12 10:17:34.000 UTC,0xa9a37b387b0017fbcb4b0c99078445f070703024,7.487485e+04,81.014588,LONG,1.830118,2323.389623,close
...,...,...,...,...,...,...,...,...,...,...
9917,wBTC,30183.908,2023-04-11 13:55:35.000 UTC,0x7b076ddcb2abe2c9c167b7f4f57e18b5f5eb1c95,4.318619e+04,51.780242,LONG,0.124933,3770.954976,close
9918,wBTC,17486.380,2023-01-10 20:21:21.000 UTC,0x10ee2ce2d6f0544ffa8222ca13878896e8124bad,2.862495e+05,326.324381,LONG,0.137138,2398.048264,close
9919,wETH,1871.830,2023-05-03 17:57:53.000 UTC,0x1c3ce81f49e2d4203bd7fa5f9c080159d566d28d,5.965176e+06,5965.175890,LONG,114.555231,214427.918101,close
9920,wETH,1909.701,2023-05-03 22:17:24.000 UTC,0x1c3ce81f49e2d4203bd7fa5f9c080159d566d28d,1.479214e+07,15798.008144,SHORT,201.647571,385086.568035,close


In [10]:
tr=pd.concat([df, df3])

tr.reset_index(drop=True, inplace=True)
tr

,pair,price,time,trader,position_side,fee,volume,token_amount,price_into_token_amount,side
0,wETH,1338.700,2023-01-10 20:37:19.000 UTC,0x8d6d43e62c30df4b3b270978316ea8c8a8daffad,SHORT,6.469667,5.711638e+03,0.294563,394.332090,open
1,wBTC,22683.000,2023-01-23 15:05:37.000 UTC,0x402030430bd2aea98edb036736db146c529aede0,SHORT,238.095238,2.380952e+05,0.220429,5000.000000,open
2,wBTC,21302.320,2022-07-26 00:03:47.000 UTC,0x35a9ebf9d86cfead962494eddab25ac8b640e9c9,LONG,122.716933,2.178973e+04,0.233495,4973.995220,open
3,wETH,1534.770,2023-03-09 00:38:38.000 UTC,0x71889f7c734342741708e9f1e0d86874c91eacaf,LONG,14.909535,1.490954e+04,0.648826,995.799348,open
4,wBTC,27246.659,2023-06-20 16:43:03.000 UTC,0x9031b0853bec292701053cea627b2ff4f22249bb,LONG,14.238571,1.423857e+04,0.010974,298.995572,open
...,...,...,...,...,...,...,...,...,...,...
28881,wBTC,30183.908,2023-04-11 13:55:35.000 UTC,0x7b076ddcb2abe2c9c167b7f4f57e18b5f5eb1c95,LONG,51.780242,4.318619e+04,0.124933,3770.954976,close
28882,wBTC,17486.380,2023-01-10 20:21:21.000 UTC,0x10ee2ce2d6f0544ffa8222ca13878896e8124bad,LONG,326.324381,2.862495e+05,0.137138,2398.048264,close
28883,wETH,1871.830,2023-05-03 17:57:53.000 UTC,0x1c3ce81f49e2d4203bd7fa5f9c080159d566d28d,LONG,5965.175890,5.965176e+06,114.555231,214427.918101,close
28884,wETH,1909.701,2023-05-03 22:17:24.000 UTC,0x1c3ce81f49e2d4203bd7fa5f9c080159d566d28d,SHORT,15798.008144,1.479214e+07,201.647571,385086.568035,close


In [11]:
tr['time'] = pd.to_datetime(tr['time'])
# tr.dtypes
tr['ts'] = tr.time.astype('int64') // 10**9
tr

,pair,price,time,trader,position_side,fee,volume,token_amount,price_into_token_amount,side,ts
0,wETH,1338.700,2023-01-10 20:37:19+00:00,0x8d6d43e62c30df4b3b270978316ea8c8a8daffad,SHORT,6.469667,5.711638e+03,0.294563,394.332090,open,1673383039
1,wBTC,22683.000,2023-01-23 15:05:37+00:00,0x402030430bd2aea98edb036736db146c529aede0,SHORT,238.095238,2.380952e+05,0.220429,5000.000000,open,1674486337
2,wBTC,21302.320,2022-07-26 00:03:47+00:00,0x35a9ebf9d86cfead962494eddab25ac8b640e9c9,LONG,122.716933,2.178973e+04,0.233495,4973.995220,open,1658793827
3,wETH,1534.770,2023-03-09 00:38:38+00:00,0x71889f7c734342741708e9f1e0d86874c91eacaf,LONG,14.909535,1.490954e+04,0.648826,995.799348,open,1678322318
4,wBTC,27246.659,2023-06-20 16:43:03+00:00,0x9031b0853bec292701053cea627b2ff4f22249bb,LONG,14.238571,1.423857e+04,0.010974,298.995572,open,1687279383
...,...,...,...,...,...,...,...,...,...,...,...
28881,wBTC,30183.908,2023-04-11 13:55:35+00:00,0x7b076ddcb2abe2c9c167b7f4f57e18b5f5eb1c95,LONG,51.780242,4.318619e+04,0.124933,3770.954976,close,1681221335
28882,wBTC,17486.380,2023-01-10 20:21:21+00:00,0x10ee2ce2d6f0544ffa8222ca13878896e8124bad,LONG,326.324381,2.862495e+05,0.137138,2398.048264,close,1673382081
28883,wETH,1871.830,2023-05-03 17:57:53+00:00,0x1c3ce81f49e2d4203bd7fa5f9c080159d566d28d,LONG,5965.175890,5.965176e+06,114.555231,214427.918101,close,1683136673
28884,wETH,1909.701,2023-05-03 22:17:24+00:00,0x1c3ce81f49e2d4203bd7fa5f9c080159d566d28d,SHORT,15798.008144,1.479214e+07,201.647571,385086.568035,close,1683152244


In [12]:
tr=tr.sort_values(by='time')
tr.reset_index(drop=True, inplace=True)

tr

,pair,price,time,trader,position_side,fee,volume,token_amount,price_into_token_amount,side,ts
0,wETH,3775.12000,2021-09-01 21:44:58+00:00,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,LONG,0.114256,114.256339,0.006000,22.650720,open,1630532698
1,wETH,3300.49000,2021-09-11 12:03:42+00:00,0x32ae17db5157cc331993376fb3ef977d28f2f158,LONG,0.065694,65.693575,0.010000,33.004900,open,1631361822
2,wETH,3286.75517,2021-09-12 02:29:02+00:00,0x076edc63cd403a0b7986f0647b7d95a0b389cb92,LONG,1.506707,1506.706587,0.230000,755.953689,open,1631413742
3,wETH,3336.99000,2021-09-12 06:14:08+00:00,0xd2776d0d1c161244667d6c58d527b927c5519c5c,LONG,0.667398,667.398000,0.100658,335.895396,open,1631427248
4,wETH,3336.99000,2021-09-12 06:15:07+00:00,0xd2776d0d1c161244667d6c58d527b927c5519c5c,LONG,1.996008,1996.007984,0.299641,999.899130,open,1631427307
...,...,...,...,...,...,...,...,...,...,...,...
28881,wETH,1848.44000,2023-07-25 00:57:17+00:00,0x5fc79d77b8c3e207cd62f7ab424a6ef5a03488ac,LONG,252.021901,252021.901430,13.503305,24960.049897,open,1690246637
28882,UNI,5.80700,2023-07-25 02:38:23+00:00,0xe0c2e7b4267b1656add89520105faf63e66d4c05,SHORT,35.006817,32244.911964,172.205958,1000.000000,open,1690252703
28883,UNI,5.76900,2023-07-25 03:50:17+00:00,0xe0c2e7b4267b1656add89520105faf63e66d4c05,LONG,80.263279,79863.959583,360.740211,2081.110280,close,1690257017
28884,LINK,7.51700,2023-07-25 06:41:11+00:00,0x03b974abcc144061eef8ed8202caed448a4abe1a,LONG,96.334661,79681.274900,2764.009478,20777.059249,close,1690267271


In [13]:
tr.to_csv('final_trades.csv')

In [12]:
tr['year'] = tr['time'].dt.year  
tr

,pair,price,time,trader,position_side,fee,volume,token_amount,price_into_token_amount,side,virtual_asset,ts,year
8433,wETH,3775.12000,2021-09-01 21:44:58+00:00,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,LONG,0.114256,114.256339,0.006000,22.650720,open,NaN,1630532698,2021
7739,wETH,3300.49000,2021-09-11 12:03:42+00:00,0x32ae17db5157cc331993376fb3ef977d28f2f158,LONG,0.065694,65.693575,0.010000,33.004900,open,NaN,1631361822,2021
5544,wETH,3286.75517,2021-09-12 02:29:02+00:00,0x076edc63cd403a0b7986f0647b7d95a0b389cb92,LONG,1.506707,1506.706587,0.230000,755.953689,open,NaN,1631413742,2021
4379,wETH,3336.99000,2021-09-12 06:14:08+00:00,0xd2776d0d1c161244667d6c58d527b927c5519c5c,LONG,0.667398,667.398000,0.100658,335.895396,open,NaN,1631427248,2021
14862,wETH,3336.99000,2021-09-12 06:15:07+00:00,0xd2776d0d1c161244667d6c58d527b927c5519c5c,LONG,1.996008,1996.007984,0.299641,999.899130,open,NaN,1631427307,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,wETH,1848.44000,2023-07-25 00:57:17+00:00,0x5fc79d77b8c3e207cd62f7ab424a6ef5a03488ac,LONG,252.021901,252021.901430,13.503305,24960.049897,open,NaN,1690246637,2023
1075,UNI,5.80700,2023-07-25 02:38:23+00:00,0xe0c2e7b4267b1656add89520105faf63e66d4c05,SHORT,35.006817,32244.911964,172.205958,1000.000000,open,NaN,1690252703,2023
21340,NaN,5.76900,2023-07-25 03:50:17+00:00,0xe0c2e7b4267b1656add89520105faf63e66d4c05,LONG,80.263279,79863.959583,360.740211,2081.110280,close,UNI,1690257017,2023
19585,NaN,7.51700,2023-07-25 06:41:11+00:00,0x03b974abcc144061eef8ed8202caed448a4abe1a,LONG,96.334661,79681.274900,2764.009478,20777.059249,close,LINK,1690267271,2023


In [9]:
new = tr[(tr['time'].dt.year < 2023) | (tr['time'].dt.month < 7)]
new.reset_index(drop=True, inplace=True)

new

,pair,time,trader,volume,fee,position_side,side,ts,year
0,wETH,2021-09-01 21:44:58+00:00,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,113.902144,0.113902,LONG,open,1630532698,2021
1,wETH,2021-09-10 21:38:09+00:00,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,113.902144,1.078198,SHORT,close,1631309889,2021
2,wETH,2021-09-11 12:03:42+00:00,0x32ae17db5157cc331993376fb3ef977d28f2f158,65.489925,0.065490,LONG,open,1631361822,2021
3,wETH,2021-09-12 02:29:02+00:00,0x076edc63cd403a0b7986f0647b7d95a0b389cb92,1502.035796,1.502036,LONG,open,1631413742,2021
4,wETH,2021-09-12 06:14:08+00:00,0xd2776d0d1c161244667d6c58d527b927c5519c5c,665.329066,0.665329,LONG,open,1631427248,2021
...,...,...,...,...,...,...,...,...,...
31421,wETH,2023-06-30 22:08:12+00:00,0xe013bf246648e8bba68c932c716f99332fcbbff8,323816.563986,323.816564,LONG,open,1688162892,2023
31422,wETH,2023-06-30 22:08:22+00:00,0x1df578bb65ce9f5f4c89cea5c19bbb840f2274e6,226082.076159,300.708881,SHORT,close,1688162902,2023
31423,wETH,2023-06-30 22:12:12+00:00,0xc3efac8177377dd2517000e9a5afa0ef4c67b200,109582.110668,109.582111,LONG,open,1688163132,2023
31424,wBTC,2023-06-30 22:12:57+00:00,0xc3efac8177377dd2517000e9a5afa0ef4c67b200,81329.547767,81.329548,LONG,open,1688163177,2023


In [10]:
sym=new['pair'].unique()
year=new['year'].unique()
mp={}
conv={'wETH': 'ETHUSDT', 'LINK': 'LINKUSDT', 'wBTC': 'BTCUSDT', 'UNI': 'UNIUSDT'}
for i in sym:
    for j in year:
        a=i+str(j)
        inp='Raw Data/'+conv[i]+'/'+str(j)+'.csv'
        print(inp)
        temp=pd.read_csv(inp)
        temp=temp.drop('Unnamed: 0', axis=1)
#         print(temp)
        mp[a]=temp

Raw Data/ETHUSDT/2021.csv
Raw Data/ETHUSDT/2022.csv
Raw Data/ETHUSDT/2023.csv
Raw Data/LINKUSDT/2021.csv
Raw Data/LINKUSDT/2022.csv
Raw Data/LINKUSDT/2023.csv
Raw Data/BTCUSDT/2021.csv
Raw Data/BTCUSDT/2022.csv
Raw Data/BTCUSDT/2023.csv
Raw Data/UNIUSDT/2021.csv
Raw Data/UNIUSDT/2022.csv
Raw Data/UNIUSDT/2023.csv


In [11]:
cnt=0
prices=[]
for i in new.index:
    sym=new['pair'][i]
    year=new['year'][i]
    data=mp[sym+str(year)]
    req=data[(abs(data['open time']/1000 - new['ts'][i]) < 30)]
    req.reset_index(drop=True, inplace=True)
    if len(req)>0:
        price=req['c'][0]+req['o'][0]
        price/=2
        price=round(price, 2)
    else:
        price=-1
    cnt+=1
    print(cnt, end='\r')
    prices.append(price)

In [12]:
new['price']=prices
new

/tmp/ipykernel_16488/3467254376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['price']=prices


,pair,time,trader,volume,fee,position_side,side,ts,year,price
0,wETH,2021-09-01 21:44:58+00:00,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,113.902144,0.113902,LONG,open,1630532698,2021,3752.71
1,wETH,2021-09-10 21:38:09+00:00,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,113.902144,1.078198,SHORT,close,1631309889,2021,3216.23
2,wETH,2021-09-11 12:03:42+00:00,0x32ae17db5157cc331993376fb3ef977d28f2f158,65.489925,0.065490,LONG,open,1631361822,2021,3300.40
3,wETH,2021-09-12 02:29:02+00:00,0x076edc63cd403a0b7986f0647b7d95a0b389cb92,1502.035796,1.502036,LONG,open,1631413742,2021,3281.75
4,wETH,2021-09-12 06:14:08+00:00,0xd2776d0d1c161244667d6c58d527b927c5519c5c,665.329066,0.665329,LONG,open,1631427248,2021,3321.09
...,...,...,...,...,...,...,...,...,...,...
31421,wETH,2023-06-30 22:08:12+00:00,0xe013bf246648e8bba68c932c716f99332fcbbff8,323816.563986,323.816564,LONG,open,1688162892,2023,1935.25
31422,wETH,2023-06-30 22:08:22+00:00,0x1df578bb65ce9f5f4c89cea5c19bbb840f2274e6,226082.076159,300.708881,SHORT,close,1688162902,2023,1935.25
31423,wETH,2023-06-30 22:12:12+00:00,0xc3efac8177377dd2517000e9a5afa0ef4c67b200,109582.110668,109.582111,LONG,open,1688163132,2023,1934.76
31424,wBTC,2023-06-30 22:12:57+00:00,0xc3efac8177377dd2517000e9a5afa0ef4c67b200,81329.547767,81.329548,LONG,open,1688163177,2023,30507.32


In [13]:
import requests

def get_binance_price(symbol, timestamp):
    base_url = 'https://api.binance.com/api/v3/klines'
    interval = '1s'  # 1-minute interval
    limit = 1       # Only retrieve 1 data point

    # Convert timestamp to milliseconds
    timestamp_ms = timestamp * 1000

    # Construct the API endpoint URL
    url = f'{base_url}?symbol={symbol}&interval={interval}&limit={limit}&startTime={timestamp_ms}'

    try:
        response = requests.get(url)
        data = response.json()
#         Extract the closing price from the response
        price = float(data[0][4])  # Closing price is at index 4

        return price
    except requests.exceptions.RequestException as e:
        print('Error occurred:', e)

In [14]:

# Example usage: Get ETHUSDT price at a specific timestamp
for i in new.index:
    if new['price'][i]==-1:
        symbol = conv[new['pair'][i]]
        timestamp = new['ts'][i]  # Replace with your desired timestamp

        price = get_binance_price(symbol, timestamp)
        print(price)
        new['price'][i]=price

3534.53


/tmp/ipykernel_16488/2444950020.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['price'][i]=price


3125.3
2928.33
26.3
26.07
24.85
27.16
61957.43
68439.76
4198.11
21.82
4403.99
4100.96
3081.94
3047.5
2930.86
3025.09
38724.09
39549.37
2450.13
2025.0
2016.69
1858.85
1734.98
30192.78
1805.95
1788.2
1260.11
24687.79
1182.54
1058.75
1043.41
1213.42
1093.18
1079.13
1078.83
1090.09
971.75
1136.75
1147.51
19556.59
1219.6
1222.39
20416.87
19710.89
20317.11
1346.71
1482.66
1482.66
1565.1
1364.7
1778.15
9.28
1575.58
1580.52
23366.48
23367.83
1690.72
1737.95
1750.73
22953.95
1681.5
1711.45
1871.64
1850.87
1870.41
1919.17
1888.91
1906.39
1885.49
1853.49
1817.18
1619.4
21497.34
1525.49
1580.52
19893.01
1570.01
1616.54
18792.4
1630.03
1641.87
21702.31
20163.35
1602.51
1586.24
1586.74
1649.88
1610.97
1578.72
1599.34
1622.39
1641.93
1634.78
1489.27
1452.08
1289.77
1327.76
1340.69
1342.45
1341.37
1241.0
1280.8
1284.12
18870.2
1394.59
20324.08
1384.03
1364.6
1330.72
19132.12
19022.92
1302.34
1354.56
19061.64
1336.47
1337.93
19692.11
1325.28
20256.35
19910.4
19113.93
1289.29
1330.2
19354.34
1328.0
1293

In [15]:
new['price'].describe()

count    31426.000000
mean      7509.703919
std      10653.657077
min          3.530000
25%       1429.335000
50%       1731.640000
75%      16472.672500
max      68469.360000
Name: price, dtype: float64

In [16]:
new.to_csv('Final_Trades_2.csv', index=False)